In [1]:
import csv
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
import warnings
import gensim
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd

## Prvi skup podataka

In [2]:
in_domain_train_tokenized = open("../dataset/cola_public_1.1/cola_public/tokenized/in_domain_train.tsv")
in_domain_dev_tokenized = open("../dataset/cola_public_1.1/cola_public/tokenized/in_domain_dev.tsv")

In [3]:
read_train = csv.reader(in_domain_train_tokenized, delimiter="\t")
train = []
for row in read_train:
    train.append(row)
in_domain_train_tokenized.close()

In [4]:
read_dev = csv.reader(in_domain_dev_tokenized, delimiter="\t")
dev = []
for row in read_dev:
    dev.append(row)
in_domain_dev_tokenized.close()

In [5]:
train_array = np.array(train)
train_df = pd.DataFrame(train_array)
train_df

,0,1,2,3
0,gj04,1,,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,,one more pseudo generalization and i 'm giving...
2,gj04,1,,one more pseudo generalization or i 'm giving ...
3,gj04,1,,"the more we study verbs , the crazier they get ."
4,gj04,1,,day by day the facts are getting murkier .
...,...,...,...,...
8546,ad03,0,*,poseidon appears to own a dragon
8547,ad03,0,*,digitize is my happiest memory
8548,ad03,1,,it is easy to slay the gorgon .
8549,ad03,1,,i had the strangest feeling that i knew you .


In [6]:
dev_array = np.array(dev)
dev_df = pd.DataFrame(dev_array)
dev_df

,0,1,2,3
0,gj04,1,,the sailors rode the breeze clear of the rocks .
1,gj04,1,,the weights made the rope stretch over the pul...
2,gj04,1,,the mechanical doll wriggled itself loose .
3,cj99,1,,"if you had eaten more , you would want less ."
4,cj99,0,*,"as you eat the most , you want the least ."
...,...,...,...,...
522,ad03,0,*,i would like to could swim
523,ad03,1,,i kicked myself
524,ad03,0,*,the bookcase ran
525,ad03,1,,i shaved myself .


In [7]:
df = pd.concat([train_df, dev_df])

In [8]:
ddf = df.drop([0, 2], axis=1)

In [9]:
prvi_df = ddf.rename(columns = {1:'Judgement', 3:'Sentence'}, inplace = False)

In [10]:
prvi_df

,Judgement,Sentence
0,1,"our friends wo n't buy this analysis , let alo..."
1,1,one more pseudo generalization and i 'm giving...
2,1,one more pseudo generalization or i 'm giving ...
3,1,"the more we study verbs , the crazier they get ."
4,1,day by day the facts are getting murkier .
...,...,...
522,0,i would like to could swim
523,1,i kicked myself
524,0,the bookcase ran
525,1,i shaved myself .


## Drugi skup podataka

In [11]:
with open('../dataset/dataset_generated.txt') as f:
    lines = f.readlines()

In [12]:
x = "no authors that the guards like have ever been popular . <eos>\n"

In [13]:
k = lines.index(x)

In [14]:
sentences1 = []
judgement1 = []
for i in range(len(lines[:k])):
    line = lines[i].split(".")
    sent = line[0]
    sentences1.append(sent)
    correct = 0
    if (i % 2) == 0:
        correct = 1
    judgement1.append(correct)

In [15]:
df1 = pd.DataFrame(list(zip(judgement1, sentences1)), columns =['Judgement', 'Sentence'])

In [16]:
df1

,Judgement,Sentence
0,1,the author that the guard likes laughs
1,0,the author that the guard likes laugh
2,1,the author that the guard likes swims
3,0,the author that the guard likes swim
4,1,the author that the guard likes smiles
...,...,...
294515,0,the bankers knew the consultants hated herself
294516,1,the bankers knew the consultants doubted thems...
294517,0,the bankers knew the consultants doubted himself
294518,1,the bankers knew the consultants doubted thems...


In [17]:
sentences2 = []
judgement2 = []
for i in range(len(lines[k:])):
    j = k + i
    line = lines[j].split(".")
    sent = line[0]
    sentences2.append(sent)
    correct = 0
    if (j % 3) == 1:
        correct = 1
    judgement2.append(correct)

In [18]:
df2 = pd.DataFrame(list(zip(judgement2, sentences2)), columns =['Judgement', 'Sentence'])

In [19]:
df2

,Judgement,Sentence
0,1,no authors that the guards like have ever been...
1,0,the authors that no guards like have ever been...
2,0,most authors that the guards like have ever be...
3,1,no authors that the guards like have ever been...
4,0,the authors that no guards like have ever been...
...,...,...
30253,0,some shows will ever be ignored
30254,0,many shows will ever be ignored
30255,1,few shows will ever get old
30256,0,some shows will ever get old


In [20]:
list_sent = sentences1 + sentences2

In [21]:
list_sent_tkn = []
for sent in list_sent:
    list_sent_tkn.append(word_tokenize(sent))

In [22]:
drugi_df = pd.concat([df1, df2])

In [23]:
drugi_df

,Judgement,Sentence
0,1,the author that the guard likes laughs
1,0,the author that the guard likes laugh
2,1,the author that the guard likes swims
3,0,the author that the guard likes swim
4,1,the author that the guard likes smiles
...,...,...
30253,0,some shows will ever be ignored
30254,0,many shows will ever be ignored
30255,1,few shows will ever get old
30256,0,some shows will ever get old


## Spajanje skupova podataka

In [24]:
kombinovani_df = pd.concat([prvi_df, drugi_df])

In [25]:
kombinovani_df

,Judgement,Sentence
0,1,"our friends wo n't buy this analysis , let alo..."
1,1,one more pseudo generalization and i 'm giving...
2,1,one more pseudo generalization or i 'm giving ...
3,1,"the more we study verbs , the crazier they get ."
4,1,day by day the facts are getting murkier .
...,...,...
30253,0,some shows will ever be ignored
30254,0,many shows will ever be ignored
30255,1,few shows will ever get old
30256,0,some shows will ever get old


## W2V model

In [26]:
# w2v_model = Word2Vec(list_sent_tkn, min_count=1, vector_size= 50, workers=3, window=3)
# words = list(w2v_model.wv.index_to_key)
# print(len(words))

In [27]:
word2vec_model = KeyedVectors.load_word2vec_format('embedding.txt', binary=False)

In [28]:
words = list(word2vec_model.index_to_key)
print(len(words))

14532


In [29]:
word_vec_dict={}
for word in words:
    word_vec_dict[word]=word2vec_model.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size

The no of key-value pairs :  14532


In [30]:
w2v_weights = word2vec_model.vectors
vocab_size, embedding_size = w2v_weights.shape
print(vocab_size, embedding_size)
MAX_SEQUENCE_LENGTH = 200

14532 300


## LSTM

In [31]:
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
import gensim
import sys
import numpy as np
import pandas as pd
import string
import logging
import random
import keras
import matplotlib.pyplot as plt

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


maxi=-1
for i,rev in enumerate(kombinovani_df['Sentence']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)


tok = Tokenizer(filters='')
tok.fit_on_texts(kombinovani_df['Sentence'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(kombinovani_df['Sentence'])

print(vocab_size)

5552


In [33]:
pad_rev= pad_sequences(encd_rev, maxlen=maxi+1, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.
print(pad_rev)
print(pad_rev.shape)

[[ 358  396  368 ...    0    0    0]
 [ 222  162 2763 ...    0    0    0]
 [ 222  162 2763 ...    0    0    0]
 ...
 [  82   92   34 ...    0    0    0]
 [ 184   92   34 ...    0    0    0]
 [ 112   92   34 ...    0    0    0]]
(333856, 45)


In [34]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embedding_size))
# print(len(tok.word_index.items()))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

train_size=0.8

y = np.array(kombinovani_df['Judgement'].tolist())

leinst = LabelEncoder()
yinst = leinst.fit_transform(y)

ycat = to_categorical(kombinovani_df['Judgement'])  # one hot target as required by NN
# print(ycat)
# X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_train, X_test, y_train, y_test = train_test_split(pad_rev, ycat, test_size=0.2, shuffle=True, random_state=42)

# test_size = 0.5
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=42)


print(X_train.shape), print(y_train.shape)
print(X_validation.shape), print(y_validation.shape)
print(X_test.shape), print(y_test.shape)

(267084, 45)
(267084, 2)
(33386, 45)
(33386, 2)
(33386, 45)
(33386, 2)


(None, None)

In [35]:
num_classes = len(leinst.classes_)
print(num_classes)
print(leinst.classes_)

2
['0' '1']


In [36]:
print(w2v_weights.shape)

(14532, 300)


In [37]:
y_validation[20:50]

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
model = Sequential()
# Keras Embedding layer with Word2Vec weights initialization
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    weights=[embed_matrix],
                    input_length=45,
                    ))

model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.add(Flatten())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, batch_size=64,
                    validation_data=(X_validation, y_validation), verbose=1)

Epoch 1/5
4174/4174 [==============================] - 368s 88ms/step - loss: 0.0750 - accuracy: 0.9532 - val_loss: 0.0155 - val_accuracy: 0.9926
Epoch 2/5
4174/4174 [==============================] - 364s 87ms/step - loss: 0.0158 - accuracy: 0.9921 - val_loss: 0.0160 - val_accuracy: 0.9922
Epoch 3/5
3052/4174 [====================>.........] - ETA: 1:47 - loss: 0.0123 - accuracy: 0.9940

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')